In [1]:
%display latex

# Manifolds

In this notebook is an abridged version of a full [SageManifolds tutorial](https://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_tutorial.ipynb), with my own additions in places.

It serves as a reference for myself, and rewritten as a way of reminding myself of the manifold features.

<!--BEGIN TOC-->
## Table of Contents
1. [Scalar fields](#0-Scalar-fields)
    1. [Action](#0-Action)
    2. [Undefined functions](#0-Undefined-functions)
    3. [Algebra](#0-Algebra)
2. [Tangent spaces](#0-Tangent-spaces)
    1. [Elements of $T_x\mathcal{M}$](#0-Elements-of-$T_x\mathcal{M}$)
3. [Vector fields](#0-Vector-fields)
    1. [Element algebra](#0-Element-algebra)
    2. [Action](#0-Action)
    3. [Components](#0-Components)
    4. [Undefined components](#0-Undefined-components)
    5. [Values at a point](#0-Values-at-a-point)

<!--END TOC-->

We will operate on the manifolds and charts defined in the previous notebook:

In [2]:
# manifolds
M = Manifold(3, 'M', latex_name=r"\mathcal{M}")

# default chart
X.<x,y,z> = M.chart()

# open subset of M
U = M.open_subset(
    'U', 
    coord_def={X: (y!=0, x<0)}
)

# chart on U
X_U = X.restrict(U)

# spherical chart
coords = r"r:(0,+oo) theta:(0,pi):\theta phi:(0,2*pi):\phi"
Y.<r,theta,phi> = U.chart(coords)

# transition maps:
xfm_Y2X_U = Y.transition_map(
    X_U,
    [
        r*sin(theta)*cos(phi), 
        r*sin(theta)*sin(phi), 
        r*cos(theta)
    ]
)
xfm_Y2X_U.set_inverse(
    sqrt(x^2+y^2+z^2), 
    atan2(sqrt(x^2+y^2),z), 
    atan2(y, x)
)

Check of the inverse coordinate transformation:
  r == r  *passed*
  theta == arctan2(r*sin(theta), r*cos(theta))  **failed**
  phi == arctan2(r*sin(phi)*sin(theta), r*cos(phi)*sin(theta))  **failed**
  x == x  *passed*
  y == y  *passed*
  z == z  *passed*
NB: a failed report can reflect a mere lack of simplification.


## Scalar fields <a name="0-Scalar-fields"></a>

A [scalar field](https://en.wikipedia.org/wiki/Scalar_field) $\varphi$ is a differentiable mapping

$$
\varphi : U \rightarrow \mathbb{R}
$$
where $U$ is an open-subset of $\mathcal{M}$.

*NB:* this is very similar to a function; and indeed, a function as we have seen so far is often called a *scalar (point) function*.

In SageManifold, we define a scalar field through:

In [3]:
f = U.scalar_field({X_U: x + y^2 + z^3}, name="f")
f.display()

f: U --> R
   (x, y, z) |--> z^3 + y^2 + x
   (r, theta, phi) |--> r^3*cos(theta)^3 + r^2*sin(phi)^2*sin(theta)^2 + r*cos(phi)*sin(theta)

Notice the field is defined in all charts in the atlas of $U$, provided there are transition maps available. To display for a specific chart, we pass it in as the argument:

In [4]:
f.display(X_U)

f: U --> R
   (x, y, z) |--> z^3 + y^2 + x

There exist a few alternative notations to the above for creating scalar fields; since we are only providing a single chart in the argument dictionary is
```py
U.scalar_field(x + y^2 + z^3, chart=X_U, name='f')
```
And indeed, since `X_U` is the default chart, it may be omitted entirely:
```py
U.scalar_field(x + y^2 + z^3, name='f')
```

### Action <a name="0-Action"></a>
The scalar field, unlike the function, will act on points in $\mathcal{M}$, not on coordinates:

In [5]:
p = M.point((1, 2, -1), name="p")
f(p)

4

We can also access the underlying function on a specific chart using the `.coord_function()` method:

In [6]:
f.coord_function(X_U).display()

(x, y, z) |--> z^3 + y^2 + x

For the symbolic expression we can use `.expr()` as usual:

In [7]:
f.expr(X_U)

z^3 + y^2 + x

Which obeys:

In [8]:
f.expr(X_U) is f.coord_function(X_U).expr()

True

### Undefined functions <a name="0-Undefined-functions"></a>
We can also define the scalar field through an unimplemented function of the chart coordinates. Consider:

In [9]:
h = U.scalar_field(function("H")(x, y, z), name="h") # default chart implicit
h.display()

h: U --> R
   (x, y, z) |--> H(x, y, z)
   (r, theta, phi) |--> H(r*cos(phi)*sin(theta), r*sin(phi)*sin(theta), r*cos(theta))

Which creates an expression when invoked:

In [10]:
h(p)

H(1, 2, -1)

We can later define an expression using chart coordinates:

In [11]:
h.set_expr(x^2 + y)
h(p)

3

Which gets mapped into the other charts also:

In [12]:
h.display()

h: U --> R
   (x, y, z) |--> x^2 + y
   (r, theta, phi) |--> r^2*cos(phi)^2*sin(theta)^2 + r*sin(phi)*sin(theta)

The `.set_expr()` method also accepts the `chart` keyword argument.

### Algebra <a name="0-Algebra"></a>
The parent of a function is the set $C^{\infty}(U)$ of all smooth scalar field on $U$. It is a commutative algebra over $\mathbb{R}$:

In [13]:
CinfU = f.parent()
print(CinfU)
CinfU, CinfU.category()

Algebra of differentiable scalar fields on the Open subset U of the 3-dimensional differentiable manifold M


(Algebra of differentiable scalar fields on the Open subset U of the 3-dimensional differentiable manifold M,
 Category of commutative algebras over Symbolic Ring)

*NB:* the base ring of the algebra is $\mathbb{R}$, which in SageMath is the Symbolic Ring (SR):

In [14]:
CinfU.base_ring()

Symbolic Ring

Arithmetic operations are defined through the algebra structure:

In [15]:
s = f + 2 * h
s.display()

U --> R
(x, y, z) |--> z^3 + 2*x^2 + y^2 + x + 2*y
(r, theta, phi) |--> r^3*cos(theta)^3 + (cos(phi)^2 + 1)*r^2*sin(theta)^2 + r*(cos(phi) + 2*sin(phi))*sin(theta)

## Tangent spaces <a name="0-Tangent-spaces"></a>
We can obtain the tangent vector space to some $p \in \mathcal{M}$ through:

In [16]:
Tp = M.tangent_space(p)
Tp

Tangent space at Point p on the 3-dimensional differentiable manifold M

The tangent space is a 2-dimensional vector space over $\mathbb{R}$ (SR):

In [17]:
print(Tp.category())
Tp.category()

Category of finite dimensional vector spaces over Symbolic Ring


Category of finite dimensional vector spaces over Symbolic Ring

Basic operations include obtaining the dimensions:

In [18]:
Tp.dim()

3

The endowed bases (deduced from the frames at $p$):

In [19]:
Tp.bases()

[Basis (d/dx,d/dy,d/dz) on the Tangent space at Point p on the 3-dimensional differentiable manifold M,
 Basis (d/dr,d/dtheta,d/dphi) on the Tangent space at Point p on the 3-dimensional differentiable manifold M]

The tangent space at $q \in \mathcal{M}, q \notin U$ has only one basis, namely the frame given by the coordinates of `X`:

In [20]:
q = M.point((1, 0, 2), name="q")

Tq = M.tangent_space(q)
Tq.bases()

[Basis (d/dx,d/dy,d/dz) on the Tangent space at Point q on the 3-dimensional differentiable manifold M]

### Elements of $T_x\mathcal{M}$ <a name="0-Elements-of-$T_x\mathcal{M}$"></a>
We can obtain a random element of the tangent space using the `.an_element()` method:

In [21]:
v = Tp.an_element()
print(v)
v.display()

Tangent vector at Point p on the 3-dimensional differentiable manifold M


d/dx + 2 d/dy + 3 d/dz

In [22]:
u = Tq.an_element()
print(u)
u.display()

Tangent vector at Point q on the 3-dimensional differentiable manifold M


d/dx + 2 d/dy + 3 d/dz

Note that desipte the similar expresion, u and v are different vectors, as they are defined on different tangent spaces, themselves defined for different points:

In [23]:
u is v

False

In [24]:
u.parent(), v.parent()

(Tangent space at Point q on the 3-dimensional differentiable manifold M,
 Tangent space at Point p on the 3-dimensional differentiable manifold M)

As such, arithmetic operations do not exist between them:

In [25]:
try:
    v + u
except TypeError as e:
    print(f"Error: {str(e)}")

Error: unsupported operand parent(s) for +: 'Tangent space at Point p on the 3-dimensional differentiable manifold M' and 'Tangent space at Point q on the 3-dimensional differentiable manifold M'


## Vector fields <a name="0-Vector-fields"></a>
Each chart defines a vector frame on the chart domain; the *coordinate basis*:

In [26]:
print(X.frame())
X.frame()

Coordinate frame (M, (d/dx,d/dy,d/dz))


Coordinate frame (M, (d/dx,d/dy,d/dz))

To get the frames defined on a manifold, we use

In [27]:
M.frames()

[Coordinate frame (M, (d/dx,d/dy,d/dz)),
 Coordinate frame (U, (d/dx,d/dy,d/dz)),
 Coordinate frame (U, (d/dr,d/dtheta,d/dphi))]

Or for the default frame (which can be set using `.set_default_frame()`), which is associated with the default chart:

In [28]:
M.default_frame()

Coordinate frame (M, (d/dx,d/dy,d/dz))

In [29]:
M.default_frame() is M.default_chart().frame()

True

Frame elements can be indexed:

In [30]:
e_frame = M.default_frame()
e2 = e_frame[2]
e2

Vector field d/dz on the 3-dimensional differentiable manifold M

### Element algebra <a name="0-Element-algebra"></a>
We can define new vector fields through:

In [31]:
v = e_frame[1] + 2 * x * e_frame[2]
v.display()

d/dy + 2*x d/dz

A vector field may be defined by components with respect to a given vector frame. If the frame is unspecified, SageManifold uses the open-set's default frame:

In [32]:
v = U.vector_field(name="v")

v[0] = 1 + y
v[1] = -x
v[2] = x * y * z

v.display()

v = (y + 1) d/dx - x d/dy + x*y*z d/dz

The above is equivalent to passing the components as arguments:

In [33]:
v = U.vector_field(1 + y, -x, x * y * z, name="v")
v.display()

v = (y + 1) d/dx - x d/dy + x*y*z d/dz

Components we do not set are assumed to be zero:

In [34]:
w = U.vector_field(name="w")

w[2] = 3

w.display()

w = 3 d/dz

Vector fields on $U$ are SageMath elements, whose parents are the set $\mathfrak{X}(U)$ of vector fields defined on $U$:

In [35]:
v.parent()

Free module X(U) of vector fields on the Open subset U of the 3-dimensional differentiable manifold M

$\mathfrak{X}(U)$ is a module over the commutative algebra $C^\infty(U)$ of scalar fields on $U$:

In [36]:
print(v.parent())

Free module X(U) of vector fields on the Open subset U of the 3-dimensional differentiable manifold M


In [37]:
v.parent().base_ring()

Algebra of differentiable scalar fields on the Open subset U of the 3-dimensional differentiable manifold M

### Action <a name="0-Action"></a>
A vector field acts on scalar fields:

In [38]:
f.display()

f: U --> R
   (x, y, z) |--> z^3 + y^2 + x
   (r, theta, phi) |--> r^3*cos(theta)^3 + r^2*sin(phi)^2*sin(theta)^2 + r*cos(phi)*sin(theta)

In [39]:
s = v(f)
s.display()

v(f): U --> R
   (x, y, z) |--> 3*x*y*z^3 - (2*x - 1)*y + 1
   (r, theta, phi) |--> -3*r^5*cos(phi)*cos(theta)^5*sin(phi) + 3*r^5*cos(phi)*cos(theta)^3*sin(phi) - 2*r^2*cos(phi)*sin(phi)*sin(theta)^2 + r*sin(phi)*sin(theta) + 1

Or component wise:

In [40]:
e_frame[2](f).display()

d/dz(f): U --> R
   (x, y, z) |--> 3*z^2
   (r, theta, phi) |--> 3*r^2*cos(theta)^2

A vector field on $U$ may be expanded in the frame associated with a chart; e.g.:

In [41]:
v.display(Y.frame())

v = (x*y*z^2 + x)/sqrt(x^2 + y^2 + z^2) d/dr - (x^3*y + x*y^3 - x)*sqrt(x^2 + y^2)*z/(x^4 + 2*x^2*y^2 + y^4 + (x^2 + y^2)*z^2) d/dtheta - (x^2 + y^2 + y)/(x^2 + y^2) d/dphi

By default, this will display in the default chart's coordinates. We can also display the vector field in different coordinates:

In [42]:
v.display(Y)

v = (r^3*cos(phi)*cos(theta)^2*sin(phi)*sin(theta)^2 + cos(phi)*sin(theta)) d/dr - (r^3*cos(phi)*cos(theta)*sin(phi)*sin(theta)^3 - cos(phi)*cos(theta))/r d/dtheta - (r*sin(theta) + sin(phi))/(r*sin(theta)) d/dphi

Which is a shorthand for

In [43]:
v.display(Y.frame(), Y)

v = (r^3*cos(phi)*cos(theta)^2*sin(phi)*sin(theta)^2 + cos(phi)*sin(theta)) d/dr - (r^3*cos(phi)*cos(theta)*sin(phi)*sin(theta)^3 - cos(phi)*cos(theta))/r d/dtheta - (r*sin(theta) + sin(phi))/(r*sin(theta)) d/dphi

### Components <a name="0-Components"></a>
Looping over the indicial components of the manifold, we can operate on the individual components of the vector field:

In [44]:
for i in M.irange():
    show(e_frame[i].display(Y))

d/dx = cos(phi)*sin(theta) d/dr + cos(phi)*cos(theta)/r d/dtheta - sin(phi)/(r*sin(theta)) d/dphi

d/dy = sin(phi)*sin(theta) d/dr + cos(theta)*sin(phi)/r d/dtheta + cos(phi)/(r*sin(theta)) d/dphi

d/dz = cos(theta) d/dr - sin(theta)/r d/dtheta

The components of a vector field w.r.t. the default frame can be obtained as a list:

In [45]:
v[:]

[y + 1, -x, x*y*z]

Or, alternatively, displayed in pretty print:

In [46]:
v.display_comp()

v^x = y + 1 
v^y = -x 
v^z = x*y*z

Which also supports alternative frames:

In [47]:
v.display_comp(Y.frame())

v^r = (x*y*z^2 + x)/sqrt(x^2 + y^2 + z^2) 
v^theta = -(x^3*y + x*y^3 - x)*sqrt(x^2 + y^2)*z/(x^4 + 2*x^2*y^2 + y^4 + (x^2 + y^2)*z^2) 
v^phi = -(x^2 + y^2 + y)/(x^2 + y^2)

We can also obtain a component list w.r.t. another frame, with the `.comp()` method:

In [48]:
v.comp(Y.frame())[:]

[(x*y*z^2 + x)/sqrt(x^2 + y^2 + z^2),
 -(x^3*y + x*y^3 - x)*sqrt(x^2 + y^2)*z/(x^4 + 2*x^2*y^2 + y^4 + (x^2 + y^2)*z^2),
 -(x^2 + y^2 + y)/(x^2 + y^2)]

Which also exists in the shortcut:

In [49]:
v[Y.frame(), :]

[(x*y*z^2 + x)/sqrt(x^2 + y^2 + z^2),
 -(x^3*y + x*y^3 - x)*sqrt(x^2 + y^2)*z/(x^4 + 2*x^2*y^2 + y^4 + (x^2 + y^2)*z^2),
 -(x^2 + y^2 + y)/(x^2 + y^2)]

We can change the coordinates to express these components in as usual with the second argument:

In [50]:
v.display_comp(Y.frame(), Y)

v^r = r^3*cos(phi)*cos(theta)^2*sin(phi)*sin(theta)^2 + cos(phi)*sin(theta) 
v^theta = -(r^3*cos(phi)*cos(theta)*sin(phi)*sin(theta)^3 - cos(phi)*cos(theta))/r 
v^phi = -(r*sin(theta) + sin(phi))/(r*sin(theta))

or with the shorthand:

To obtain the components as scalar fields, we can use the double square bracket syntax:

In [51]:
v[[0]].display() # x component as scalar field

U --> R
(x, y, z) |--> y + 1
(r, theta, phi) |--> r*sin(phi)*sin(theta) + 1

Or find the symbolic expression represented on another chart:

In [52]:
v[[0]].expr(Y)

r*sin(phi)*sin(theta) + 1

### Undefined components <a name="0-Undefined-components"></a>
We can even define unspecified functional components of vector fields:

In [53]:
u = U.vector_field(name="u")
u[:] = [
    function("u_x")(x, y, z),
    function("u_y")(x, y, z),
    function("u_z")(x, y, z)
]
u.display()

u = u_x(x, y, z) d/dx + u_y(x, y, z) d/dy + u_z(x, y, z) d/dz

which still obeys the algebra:

In [54]:
s = v + u
s.set_name("s")
s.display()

s = (y + u_x(x, y, z) + 1) d/dx + (-x + u_y(x, y, z)) d/dy + (x*y*z + u_z(x, y, z)) d/dz

### Values at a point <a name="0-Values-at-a-point"></a>
The value of a vector field at some point $p \in \mathcal{M}$ is obtained with the `.at()` member function:

In [55]:
vp = v.at(p)
vp.display()

v = 3 d/dx - d/dy - 2 d/dz

Which has used the components of $p$ in `X_U`:

In [56]:
p.coord(X_U)

(1, 2, -1)

To clarify ambiguities, it can be useful to set the name of the vector field:

In [57]:
vp.set_name(latex_name="v|_p")
vp.display()

v = 3 d/dx - d/dy - 2 d/dz

By contrast, using the generic vector field:

In [58]:
up = u.at(p)
up.set_name(latex_name="u|_p")

up.display()

u = u_x(1, 2, -1) d/dx + u_y(1, 2, -1) d/dy + u_z(1, 2, -1) d/dz